In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from scipy.special import expit as sigmoid
from scipy.stats import multivariate_normal

In [18]:
[M, N] = [10,20]

sigma =2
y=np.random.binomial(n=1, p=0.5,size=(10,20)) + np.random.randn(10,20)
J=1
rate=0.5
max_iter=15
ELBO= np.zeros(max_iter)

Hx_mean=np.zeros(max_iter)

In [26]:
logodds = multivariate_normal.logpdf(y.flatten(), mean=+1, cov=sigma**2)-multivariate_normal.logpdf(y.flatten(), mean=-1, cov=sigma**2) 
logodds = np.reshape(logodds, (M,N))

In [29]:
p1 = sigmoid(logodds)
mu = 2*p1 -1

In [37]:
a = mu + 0.5*logodds
qxp1 = sigmoid(2*a)
qxm1 = sigmoid(-2*a)

In [57]:
logp1 = np.reshape(multivariate_normal.logpdf(y.flatten(), mean=+1, cov=sigma**2), (M,N))
logm1 = np.reshape(multivariate_normal.logpdf(y.flatten(), mean=-1, cov=sigma**2), (M,N))

In [62]:
max_iter=1

In [63]:
for i in tqdm(range(max_iter)):
    muNew = mu
    for ix in range(N):
        for iy in range(M):
            pos = iy+M*ix
            neighborhood = pos + np.array([-1,1,-M,M])
            boundary_idx = [iy!=0, iy!=M-1, ix!=0, ix!=N-1]
            neighborhood = neighborhood[np.where(boundary_idx)[0]]
            xx, yy = np.unravel_index(pos, (M,N), order='F')
            nx, ny = np.unravel_index(neighborhood, (M,N), order='F')
            
            Sbar = J*np.sum(mu[nx,ny])
            muNew[xx,yy] = (1-rate)*muNew[xx,yy] + rate*np.tanh(Sbar + 0.5 * logodds[xx,yy])
            ELBO[i] = ELBO[i] + 0.5*(Sbar+muNew[xx,yy])
            
            print(xx,yy)
            print(nx,ny)
    mu = muNew
    
    a = mu + 0.5*(logodds)
    qxp1 = sigmoid(2*a)
    qxm1 = sigmoid(-2*a)
    Hx = -qxm1*np.log(qxm1+1e-10) - qxp1*np.log(qxp1+1e-10)
    
    ELBO[i] = ELBO[i] + np.sum(qxp1*logp1 + qxm1*logm1) + np.sum(Hx)
    Hx_mean[i] = np.mean(Hx)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

0 0
[1 0] [0 1]
1 0
[0 2 1] [0 0 1]
2 0
[1 3 2] [0 0 1]
3 0
[2 4 3] [0 0 1]
4 0
[3 5 4] [0 0 1]
5 0
[4 6 5] [0 0 1]
6 0
[5 7 6] [0 0 1]
7 0
[6 8 7] [0 0 1]
8 0
[7 9 8] [0 0 1]
9 0
[8 9] [0 1]
0 1
[1 0 0] [1 0 2]
1 1
[0 2 1 1] [1 1 0 2]
2 1
[1 3 2 2] [1 1 0 2]
3 1
[2 4 3 3] [1 1 0 2]
4 1
[3 5 4 4] [1 1 0 2]
5 1
[4 6 5 5] [1 1 0 2]
6 1
[5 7 6 6] [1 1 0 2]
7 1
[6 8 7 7] [1 1 0 2]
8 1
[7 9 8 8] [1 1 0 2]
9 1
[8 9 9] [1 0 2]
0 2
[1 0 0] [2 1 3]
1 2
[0 2 1 1] [2 2 1 3]
2 2
[1 3 2 2] [2 2 1 3]
3 2
[2 4 3 3] [2 2 1 3]
4 2
[3 5 4 4] [2 2 1 3]
5 2
[4 6 5 5] [2 2 1 3]
6 2
[5 7 6 6] [2 2 1 3]
7 2
[6 8 7 7] [2 2 1 3]
8 2
[7 9 8 8] [2 2 1 3]
9 2
[8 9 9] [2 1 3]
0 3
[1 0 0] [3 2 4]
1 3
[0 2 1 1] [3 3 2 4]
2 3
[1 3 2 2] [3 3 2 4]
3 3
[2 4 3 3] [3 3 2 4]
4 3
[3 5 4 4] [3 3 2 4]
5 3
[4 6 5 5] [3 3 2 4]
6 3
[5 7 6 6] [3 3 2 4]
7 3
[6 8 7 7] [3 3 2 4]
8 3
[7 9 8 8] [3 3 2 4]
9 3
[8 9 9] [3 2 4]
0 4
[1 0 0] [4 3 5]
1 4
[0 2 1 1] [4 4 3 5]
2 4
[1 3 2 2] [4 4 3 5]
3 4
[2 4 3 3] [4 4 3 5]
4 4
[3 5 4 4] [4 4 3

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.11s/it]


In [56]:
print(nx)
print(ny)

[8 9]
[19 18]
